In [1]:
# put absolute path here
DATA_DIR = r"C:\Users\Joanna\NLP_homeworks\json"

In [2]:
# making usage of my helper .py file in jupyter environment on any computer
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from file_utils import *
data_manager = DataManager(DATA_DIR)

In [3]:
from elasticsearch_dsl import connections, InnerDoc, Text, DocType, Date, Keyword, Nested, Index, analyzer

In [4]:
connections.create_connection(hosts=['localhost'], timeout=20)

judgments = Index(name='judgments')
analyzer = analyzer(
    "my_morfologik",
    tokenizer="standard",
    filter=["morfologik_stem"],
    analyzer="morfologik"
)
judgments.analyzer(analyzer)
judgments.delete(ignore=404)
judgments.create()

In [5]:
class Judge(InnerDoc):
    name = Text()

class Judgment(DocType):
    text_field = Text(analyzer=analyzer)
    date = Date()
    signature = Keyword()
    judges = Nested(Judge)
    
    def add_judge(self, name):
        self.judges.append(Judge(name=name))
        
    class Meta:
        index = 'judgments'
        
judgments.doc_type(Judgment)        

__main__.Judgment

In [6]:
generator = data_manager.judgments_generator(year=2009)
while True:
    try:  
        content = generator.__next__()
        text = content["textContent"]
        date = content["judgmentDate"]
        signature = content["id"]
        
        judgment = Judgment(text_field=text, date=date, signature=signature, judges=[])
        for judge in content["judges"]:
            judgment.add_judge(name=judge['name'])
        judgment.save()
    except StopIteration:
        print("Processing finished")
        break

Processing finished


In [7]:
from elasticsearch_dsl import Search
query = Search().query("match", text_field="szkoda")
query.execute()
print("Total number of judgments cotaining damage word is: %d" % query.count())

Total number of judgments cotaining damage word is: 265


In [8]:
#TODO: Make work for 'trwały uszczerbek na zdrowie'!
query = Search().query("match_phrase", text_field="trwały uszczerbek na zdrowiu")
query.execute()
print("Total number of judgments cotaining phrase 'trwały uszczerbek na zdrowiu': %d" % query.count())

Total number of judgments cotaining phrase 'trwały uszczerbek na zdrowiu': 4


In [10]:
query = Search().query("span_near", 
                       clauses=[
                           {"span_term":{"text_field":"trwały"}},
                           {"span_term":{"text_field":"uszczerbek"}},
                           {"span_term":{"text_field":"na"}},
                           {"span_term":{"text_field":"zdrowiu"}}
                        ],
                       slop=2,
                       in_order= True)
query.execute()
print("Total number of judgments containing phrase 'trwały uszczerbek na zdrowiu' with span: %d" % query.count())

Total number of judgments containing phrase 'trwały uszczerbek na zdrowiu' with span: 6
